残差连接层

In [3]:
import torch


class Resnet(torch.nn.Module):

    def __init__(self, dim_in, dim_out):
        super().__init__()

        self.time = torch.nn.Sequential(
            torch.nn.SiLU(),
            torch.torch.nn.Linear(1280, dim_out),
            torch.nn.Unflatten(dim=1, unflattened_size=(dim_out, 1, 1)),
        )

        self.s0 = torch.nn.Sequential(
            torch.torch.nn.GroupNorm(num_groups=32,
                                     num_channels=dim_in,
                                     eps=1e-05,
                                     affine=True),
            torch.nn.SiLU(),
            torch.torch.nn.Conv2d(dim_in,
                                  dim_out,
                                  kernel_size=3,
                                  stride=1,
                                  padding=1),
        )

        self.s1 = torch.nn.Sequential(
            torch.torch.nn.GroupNorm(num_groups=32,
                                     num_channels=dim_out,
                                     eps=1e-05,
                                     affine=True),
            torch.nn.SiLU(),
            torch.torch.nn.Conv2d(dim_out,
                                  dim_out,
                                  kernel_size=3,
                                  stride=1,
                                  padding=1),
        )

        self.res = None
        if dim_in != dim_out:
            self.res = torch.torch.nn.Conv2d(dim_in,
                                             dim_out,
                                             kernel_size=1,
                                             stride=1,
                                             padding=0)

    def forward(self, x, time):
        #x -> [1, 320, 32, 32]
        #time -> [1, 1280]

        res = x

        #[1, 1280] -> [1, 640, 1, 1]
        time = self.time(time)

        #[1, 320, 32, 32] -> [1, 640, 32, 32]
        x = self.s0(x) + time

        #维度不变
        #[1, 640, 32, 32]
        x = self.s1(x)

        #[1, 320, 64, 64] -> [1, 640, 32, 32]
        if self.res:
            res = self.res(res)

        #维度不变
        #[1, 640, 32, 32]
        x = res + x

        return x


Resnet(320, 640)(torch.randn(1, 320, 32, 32), torch.randn(1, 1280)).shape

torch.Size([1, 640, 32, 32])

UNet的注意力层

In [4]:
class CrossAttention(torch.nn.Module):
    """
    初始化部分
    """
    def __init__(self, dim_q, dim_kv):
        #dim_q -> 320
        #dim_kv -> 768

        super().__init__()

        self.dim_q = dim_q

        self.q = torch.nn.Linear(dim_q, dim_q, bias=False)
        self.k = torch.nn.Linear(dim_kv, dim_q, bias=False)
        self.v = torch.nn.Linear(dim_kv, dim_q, bias=False)

        self.out = torch.nn.Linear(dim_q, dim_q)

    """
    图文注意力
    计算过程
    无mask
    多头注意力
    q是图像数据，kv是文本数据，q，kv不相等，计算的是交叉注意力
    """
    def forward(self, q, kv):
        #q -> [1, 4096, 320]
        #kv -> [1, 77, 768]

        #[1, 4096, 320] -> [1, 4096, 320]
        q = self.q(q)
        #[1, 77, 768] -> [1, 77, 320]
        k = self.k(kv)
        #[1, 77, 768] -> [1, 77, 320]
        v = self.v(kv)

        def reshape(x):
            #x -> [1, 4096, 320]
            b, lens, dim = x.shape

            #[1, 4096, 320] -> [1, 4096, 8, 40]
            x = x.reshape(b, lens, 8, dim // 8)

            #[1, 4096, 8, 40] -> [1, 8, 4096, 40]
            x = x.transpose(1, 2)

            #[1, 8, 4096, 40] -> [8, 4096, 40]
            x = x.reshape(b * 8, lens, dim // 8)

            return x

        #[1, 4096, 320] -> [8, 4096, 40]
        q = reshape(q)
        #[1, 77, 320] -> [8, 77, 40]
        k = reshape(k)
        #[1, 77, 320] -> [8, 77, 40]
        v = reshape(v)

        #[8, 4096, 40] * [8, 40, 77] -> [8, 4096, 77]
        #atten = q.bmm(k.transpose(1, 2)) * (self.dim_q // 8)**-0.5

        #从数学上是等价的,但是在实际计算时会产生很小的误差
        atten = torch.baddbmm(
            torch.empty(q.shape[0], q.shape[1], k.shape[1], device=q.device),
            q,
            k.transpose(1, 2),
            beta=0,
            alpha=(self.dim_q // 8)**-0.5,
        )

        atten = atten.softmax(dim=-1)

        #[8, 4096, 77] * [8, 77, 40] -> [8, 4096, 40]
        atten = atten.bmm(v)

        def reshape(x):
            #x -> [8, 4096, 40]
            b, lens, dim = x.shape

            #[8, 4096, 40] -> [1, 8, 4096, 40]
            x = x.reshape(b // 8, 8, lens, dim)

            #[1, 8, 4096, 40] -> [1, 4096, 8, 40]
            x = x.transpose(1, 2)

            #[1, 4096, 320]
            x = x.reshape(b // 8, lens, dim * 8)

            return x

        #[8, 4096, 40] -> [1, 4096, 320]
        atten = reshape(atten)

        #[1, 4096, 320] -> [1, 4096, 320]
        atten = self.out(atten)

        return atten


CrossAttention(320, 768)(torch.randn(1, 4096, 320), torch.randn(1, 77,
                                                                768)).shape

torch.Size([1, 4096, 320])

Transformer层

计算过程

这是一个 U-Net/扩散模型里常见的 Transformer 块（很多人称它为 Transformer ResBlock 或 Spatial Transformer）：
- In：对 [B, C, H, W] 归一化+1×1卷积 → 拉平成 [B, HW, C] 序列；
- Self-Attn：图像 token 彼此交互；
- Cross-Attn：图像 token 与文本 token 交互（多模态对齐，常见于文生图、条件扩散、开放词汇分割）；
- MLP(GEGLU)：非线性前馈增强表达；
- Out：把序列折回 2D，并与输入特征残差融合。
全流程均为 Pre-Norm + 残差 设计，训练更稳定、梯度更顺畅。

In [5]:
class Transformer(torch.nn.Module):

    def __init__(self, dim): # dim 是通道/嵌入维度
        super().__init__()

        self.dim = dim

        """
        In 分支：把 2D 特征变成 token 序列前的归一化+1×1卷积
        """
        #in
        self.norm_in = torch.nn.GroupNorm(num_groups=32,
                                          num_channels=dim,
                                          eps=1e-6,
                                          affine=True)
        self.cnn_in = torch.nn.Conv2d(dim,
                                      dim,
                                      kernel_size=1,
                                      stride=1,
                                      padding=0)

        """
        注意力块：先自注意力（图像内部），再跨注意力（图像对文本）
        atten1：q 与 kv 都来自同一图像 token 序列（自注意力）。
        atten2：q 来自图像，kv 是外部文本特征（77 个 token，维度 768），做跨模态对齐。
        """
        #atten
        self.norm_atten0 = torch.nn.LayerNorm(dim, elementwise_affine=True)
        self.atten1 = CrossAttention(dim, dim)
        self.norm_atten1 = torch.nn.LayerNorm(dim, elementwise_affine=True)
        self.atten2 = CrossAttention(dim, 768)

        """
        激活/MLP（门控前馈 GEGLU 风格）
        """
        #act
        self.norm_act = torch.nn.LayerNorm(dim, elementwise_affine=True)
        self.fc0 = torch.nn.Linear(dim, dim * 8)
        self.act = torch.nn.GELU()
        self.fc1 = torch.nn.Linear(dim * 4, dim)

        """
        Out 分支：把 token 序列还原回 2D，再做 1×1 卷积并加上输入残差
        """
        #out
        self.cnn_out = torch.nn.Conv2d(dim,
                                       dim,
                                       kernel_size=1,
                                       stride=1,
                                       padding=0)

    def forward(self, q, kv):
        #q -> [1, 320, 64, 64]
        #kv -> [1, 77, 768]
        b, _, h, w = q.shape
        res1 = q

        #----in----
        #维度不变
        #[1, 320, 64, 64]
        q = self.cnn_in(self.norm_in(q))

        #[1, 320, 64, 64] -> [1, 64, 64, 320] -> [1, 4096, 320]
        q = q.permute(0, 2, 3, 1).reshape(b, h * w, self.dim)

        #----atten----
        #维度不变
        #[1, 4096, 320]
        """
        1) 自注意力（图像内部）
        先 LayerNorm，再把同一个归一化后的 q 同时作为 q 和 kv 喂给 atten1（self-attn）。
        残差连接：输出 + q。这一步让图像 token 之间互相“沟通”。
        """
        q = self.atten1(q=self.norm_atten0(q), kv=self.norm_atten0(q)) + q
        """
        2) 跨注意力（图像对文本）
        再次 LayerNorm 后，把图像 token 作为查询，文本 token（[B, 77, 768]）作为键值做 cross-attn。
        CrossAttention(dim, 768) 内部会把文本 kv 投影到 dim=320 再与 q 匹配。
        残差相加：跨模态融合信息（图像特征可“对齐/检索”文本概念）。
        """
        q = self.atten2(q=self.norm_atten1(q), kv=kv) + q

        #----act----
        """
        fc0 把每个 token 维度从 320 放大到 2560 (= 8*dim)。
        对半切分为 x1, x2（各 1280），做 x1 * GELU(x2) → 门控激活（GEGLU 思想）。
        fc1 把 1280 压回 320，并与 res2 残差相加。
        （这一段相当于 Transformer 里的 FFN，只是用了更强的门控变体。）
        """
        #[1, 4096, 320]
        res2 = q

        #[1, 4096, 320] -> [1, 4096, 2560]
        q = self.fc0(self.norm_act(q))

        #1280
        d = q.shape[2] // 2

        #[1, 4096, 1280] * [1, 4096, 1280] -> [1, 4096, 1280]
        q = q[:, :, :d] * self.act(q[:, :, d:])

        #[1, 4096, 1280] -> [1, 4096, 320]
        q = self.fc1(q) + res2

        #----out----
        #[1, 4096, 320] -> [1, 64, 64, 320] -> [1, 320, 64, 64]
        q = q.reshape(b, h, w, self.dim).permute(0, 3, 1, 2).contiguous() # 把 token 序列重新“折叠”回特征图。

        """
        1×1 Conv 再做一次通道融合，然后与最初输入的 2D 特征做残差加和（res1）。
        返回形状与输入完全一致：[1, 320, 64, 64]。
        """
        #维度不变
        #[1, 320, 64, 64]
        q = self.cnn_out(q) + res1

        return q


Transformer(320)(torch.randn(1, 320, 64, 64), torch.randn(1, 77, 768)).shape

torch.Size([1, 320, 64, 64])

Down层

这个 DownBlock 在网络里的角色

通道升维 + 条件注入（time）：让更深层有更强表征，并把扩散时间步条件灌进来。

两次 (Res→TF)：在同一分辨率下先充分空间建模 + 文本对齐，避免信息还没融合就被下采样丢细节。

下采样：把空间分辨率减半，进入更抽象的层级，同时把多尺度特征存入 outs 供解码端跳连。

In [9]:
class DownBlock(torch.nn.Module):

    def __init__(self, dim_in, dim_out):
        super().__init__()

        self.tf0 = Transformer(dim_out)
        self.res0 = Resnet(dim_in, dim_out)

        self.tf1 = Transformer(dim_out)
        self.res1 = Resnet(dim_out, dim_out)

        self.out = torch.nn.Conv2d(dim_out,
                                   dim_out,
                                   kernel_size=3,
                                   stride=2,
                                   padding=1)

    """
    输入：
    out_vae: 图像特征 [1, 320, 32, 32]
    out_encoder: 文本/编码器特征 [1, 77, 768]
    time: 时间嵌入 [1, 1280]（常见做法：先经 MLP 映射，再注入到 ResNet 内）
    """
    def forward(self, out_vae, out_encoder, time):
        outs = [] # 准备收集本层的跳连特征（给 UNet 上采样端用）。

        """
        保存第一个跳连：outs[0] = [1, 320, 32, 32] → [1, 640, 32, 32]
        """
        out_vae = self.res0(out_vae, time) # 通道升维 + 融合 time 条件（如 FiLM/加性/缩放偏置等）。
        out_vae = self.tf0(out_vae, out_encoder) # 先自注意力建模空间 token 间关系，再跨注意力用 out_encoder（77×768）为键值增强语义（图像对文本对齐/检索）。
        outs.append(out_vae) # 保存第一个跳连：outs[0] = [1, 640, 32, 32]

        out_vae = self.res1(out_vae, time) # 同样融合 time，但不改通道数。
        out_vae = self.tf1(out_vae, out_encoder) # 再做一轮自/跨注意力，加强多模态语义与空间关系。
        outs.append(out_vae) # 保存第二个跳连：outs[1] = [1, 640, 32, 32]

        out_vae = self.out(out_vae)
        """
        这是 Conv2d(640→640, k3, s2, p1)：
        形状：[1, 640, 32, 32] → [1, 640, 16, 16]（H、W 各减半）
        作用：进入更低分辨率层级，同时保持通道数。
        """
        outs.append(out_vae)
        """
        保存第三个特征：outs[2] = [1, 640, 16, 16]
        许多实现里会把下采样前和/或下采样后的特征都存起来，方便解码端不同分辨率的 skip 使用。你这里三处都存了。
        """
        
        """
        out_vae：下采样后的输出，形状 [1, 640, 16, 16]（你在代码末尾打印的就是这个）
        outs：长度为 3 的列表：
        outs[0] = [1, 640, 32, 32]（第一组 Res+TF 之后）
        outs[1] = [1, 640, 32, 32]（第二组 Res+TF 之后）
        outs[2] = [1, 640, 16, 16]（下采样卷积之后）
        """
        return out_vae, outs


DownBlock(320, 640)(torch.randn(1, 320, 32, 32), torch.randn(1, 77, 768),
                    torch.randn(1, 1280))[0].shape

torch.Size([1, 640, 16, 16])

Up层

带跨层连接

三次“拼接→ResNet→Transformer”：
逐步融合来自编码端/下采样端的三路 skip 特征（两路 640 通道 + 一路 320 通道），每次拼接后用 ResNet 把通道压回到统一的 640，再用 Transformer 与 out_encoder（如文本）做跨模态交互（cross-attn），增强语义对齐。

pop() 顺序：
从尾部拿，保证与下采样端对称（编码时 append 的最后一个，解码时最先用），分辨率上这三路都和当前 out_vae 对齐（都是 32×32），因此能直接 cat。

time 向量 [1,1280]：
残差块内部通常会把时间嵌入（或噪声/步数嵌入）投影到通道维度后加到特征里；因此 time 的原始长度和输出通道数不同没关系，模块内部会线性映射。

上采样位置：
UpBlock 的语义是“解码/上采样”阶段的一个层级。设置 add_up=True 时，这个块负责把空间尺度放大一倍，为更高分辨率的下一层做准备。

In [10]:
class UpBlock(torch.nn.Module):

    def __init__(self, dim_in, dim_out, dim_prev, add_up): # 这里 dim_in=320, dim_out=640, dim_prev=1280, add_up=True。
        super().__init__()

        self.res0 = Resnet(dim_out + dim_prev, dim_out) # 第一个残差块（ResNet block），输入通道数是 dim_out + dim_prev = 640 + 1280 = 1920，输出通道数是 dim_out = 640。
                                                        # 用意：先把当前主支路(out_vae，通道 1280)与第1个跳连特征拼接后（通道加起来），再压回到 640 通道。
        self.res1 = Resnet(dim_out + dim_out, dim_out) # 第二个残差块，输入 640 + 640 = 1280，输出 640。
                                                       # 用意：再和第二个跳连特征（640 通道）拼接→压回 640。
        self.res2 = Resnet(dim_in + dim_out, dim_out) # 第三个残差块，输入 dim_in + dim_out = 320 + 640 = 960，输出 640。
                                                      # 用意：最后和第三个跳连特征（320 通道）拼接→压回 640。

        """
        三个 Transformer（通常是跨注意力 cross-attn 到 out_encoder 文本/编码器特征），通道不变，均保持 640。
        """
        self.tf0 = Transformer(dim_out)
        self.tf1 = Transformer(dim_out)
        self.tf2 = Transformer(dim_out)

        """
        若 add_up=True，最后会上采样 x2（nearest）到更大的空间分辨率，然后再一个 3×3 卷积（通道数仍 640）。
        """
        self.out = None
        if add_up:
            self.out = torch.nn.Sequential(
                torch.nn.Upsample(scale_factor=2, mode='nearest'),
                torch.nn.Conv2d(dim_out, dim_out, kernel_size=3, padding=1),
            )

    def forward(self, out_vae, out_encoder, time, out_down):
        out_vae = self.res0(torch.cat([out_vae, out_down.pop()], dim=1), time) # list.pop() 是从末尾弹出，因此 3 次 pop() 的顺序依次得到
        """
        先拼接: out_vae [1,1280,32,32] 与 pop()[1,640,32,32] → [1, 1920, 32, 32]
        过 Resnet(1920→640) → out_vae [1, 640, 32, 32]
        """
        out_vae = self.tf0(out_vae, out_encoder) # Transformer 保持通道不变 → [1, 640, 32, 32]

        out_vae = self.res1(torch.cat([out_vae, out_down.pop()], dim=1), time)
        """
        # 拼接: [1,640,32,32] 与 pop()[1,640,32,32] → [1, 1280, 32, 32]
        # Resnet(1280→640) → [1, 640, 32, 32]
        """
        out_vae = self.tf1(out_vae, out_encoder) # Transformer 保持不变 → [1, 640, 32, 32]

        out_vae = self.res2(torch.cat([out_vae, out_down.pop()], dim=1), time)
        """
        # 拼接: [1,640,32,32] 与 pop()[1,320,32,32] → [1, 960, 32, 32]
        # Resnet(960→640) → [1, 640, 32, 32]
        """
        out_vae = self.tf2(out_vae, out_encoder) # Transformer 保持不变 → [1, 640, 32, 32]

        if self.out:
            out_vae = self.out(out_vae)
        # Upsample(scale_factor=2): [1, 640, 32, 32] → [1, 640, 64, 64]
        # Conv3x3(640→640, padding=1): 形状不变 → [1, 640, 64, 64]

        return out_vae


UpBlock(320, 640, 1280, True)(torch.randn(1, 1280, 32, 32),
                              torch.randn(1, 77, 768), torch.randn(1, 1280), [
                                  torch.randn(1, 320, 32, 32),
                                  torch.randn(1, 640, 32, 32),
                                  torch.randn(1, 640, 32, 32)
                              ]).shape

torch.Size([1, 640, 64, 64])

整体结构速览（UNet 做了什么）
- 输入侧（in）：
  - in_vae: Conv2d(4→320) 把 [B,4,64,64] 提升到 [B,320,64,64]
  - in_time: Linear(320→1280→1280) 把时间嵌入弄到通道 1280
- 下采样（down）：三个 DownBlock
    - DownBlock(320→320)：输出 [B,320,32,32]
    - DownBlock(320→640)：输出 [B,640,16,16]
    - DownBlock(640→1280)：输出 [B,1280,8,8]
    - 额外两层 Resnet(1280→1280)（都在 8×8），并且把中间特征一路 append 到 out_down，供后面 U 形结构的跳连使用
- 中间（mid）：
  - Resnet(1280) → Transformer(1280) → Resnet(1280)，都在 8×8
- 上采样（up）：
  - 三个先拼接再 Resnet的块：up_res0/up_res1/up_res2，每次都把当前特征 [B,1280,8,8] 与 out_down.pop()（也是 [B,1280,8,8]）拼，一起过 Resnet(2560→1280)
  - up_in 上采样 8→16，保持通道 1280
  - 三个 UpBlock（每个内部都是三次“拼接→Resnet→Transformer”，并按 add_up 决定是否再上采样）：
    - UpBlock(640, 1280, 1280, True)：输出 [B,1280,32,32]
    - UpBlock(320, 640, 1280, True)：输出 [B,640,64,64]
    - UpBlock(320, 320, 640, False)：输出 [B,320,64,64]
- 输出侧（out）：
  - GroupNorm(320, groups=32) + SiLU + Conv2d(320→4)：最终 [B,4,64,64]

UNet模型

初始化部分

forward函数
1层in （卷积特征图 编码time）

4层down （计算图文注意力 一层一层计算 保留全部计算结果）

1层middle （计算图文注意力）

4层up （跨层链接down 一层一层计算）

1层out （卷积后输出）

In [12]:
class UNet(torch.nn.Module):

    def __init__(self):
        super().__init__()

        #in
        self.in_vae = torch.nn.Conv2d(4, 320, kernel_size=3, padding=1)

        self.in_time = torch.nn.Sequential(
            torch.nn.Linear(320, 1280),
            torch.nn.SiLU(),
            torch.nn.Linear(1280, 1280),
        )

        #down
        self.down_block0 = DownBlock(320, 320)
        self.down_block1 = DownBlock(320, 640)
        self.down_block2 = DownBlock(640, 1280)

        self.down_res0 = Resnet(1280, 1280)
        self.down_res1 = Resnet(1280, 1280)

        #mid
        self.mid_res0 = Resnet(1280, 1280)
        self.mid_tf = Transformer(1280)
        self.mid_res1 = Resnet(1280, 1280)

        #up
        self.up_res0 = Resnet(2560, 1280)
        self.up_res1 = Resnet(2560, 1280)
        self.up_res2 = Resnet(2560, 1280)

        self.up_in = torch.nn.Sequential(
            torch.nn.Upsample(scale_factor=2, mode='nearest'),
            torch.nn.Conv2d(1280, 1280, kernel_size=3, padding=1),
        )

        self.up_block0 = UpBlock(640, 1280, 1280, True)
        self.up_block1 = UpBlock(320, 640, 1280, True)
        self.up_block2 = UpBlock(320, 320, 640, False)

        #out
        self.out = torch.nn.Sequential(
            torch.nn.GroupNorm(num_channels=320, num_groups=32, eps=1e-5),
            torch.nn.SiLU(),
            torch.nn.Conv2d(320, 4, kernel_size=3, padding=1),
        )

    def forward(self, out_vae, out_encoder, time):
        #out_vae -> [1, 4, 64, 64]
        #out_encoder -> [1, 77, 768]
        #time -> [1]

        #----in----
        #[1, 4, 64, 64] -> [1, 320, 64, 64]
        out_vae = self.in_vae(out_vae)

        def get_time_embed(t):
            #-9.210340371976184 = -math.log(10000)
            e = torch.arange(160) * -9.210340371976184 / 160
            e = e.exp().to(t.device) * t

            #[160+160] -> [320] -> [1, 320]
            e = torch.cat([e.cos(), e.sin()]).unsqueeze(dim=0)

            return e

        #[1] -> [1, 320]
        time = get_time_embed(time)
        #[1, 320] -> [1, 1280]
        time = self.in_time(time)

        #----down----
        #[1, 320, 64, 64]
        #[1, 320, 64, 64]
        #[1, 320, 64, 64]
        #[1, 320, 32, 32]
        #[1, 640, 32, 32]
        #[1, 640, 32, 32]
        #[1, 640, 16, 16]
        #[1, 1280, 16, 16]
        #[1, 1280, 16, 16]
        #[1, 1280, 8, 8]
        #[1, 1280, 8, 8]
        #[1, 1280, 8, 8]
        out_down = [out_vae]

        #[1, 320, 64, 64],[1, 77, 768],[1, 1280] -> [1, 320, 32, 32]
        #out -> [1, 320, 64, 64],[1, 320, 64, 64][1, 320, 32, 32]
        out_vae, out = self.down_block0(out_vae=out_vae,
                                        out_encoder=out_encoder,
                                        time=time)
        out_down.extend(out)

        #[1, 320, 32, 32],[1, 77, 768],[1, 1280] -> [1, 640, 16, 16]
        #out -> [1, 640, 32, 32],[1, 640, 32, 32],[1, 640, 16, 16]
        out_vae, out = self.down_block1(out_vae=out_vae,
                                        out_encoder=out_encoder,
                                        time=time)
        out_down.extend(out)

        #[1, 640, 16, 16],[1, 77, 768],[1, 1280] -> [1, 1280, 8, 8]
        #out -> [1, 1280, 16, 16],[1, 1280, 16, 16],[1, 1280, 8, 8]
        out_vae, out = self.down_block2(out_vae=out_vae,
                                        out_encoder=out_encoder,
                                        time=time)
        out_down.extend(out)

        #[1, 1280, 8, 8],[1, 1280] -> [1, 1280, 8, 8]
        out_vae = self.down_res0(out_vae, time)
        out_down.append(out_vae)

        #[1, 1280, 8, 8],[1, 1280] -> [1, 1280, 8, 8]
        out_vae = self.down_res1(out_vae, time)
        out_down.append(out_vae)

        #----mid----
        #[1, 1280, 8, 8],[1, 1280] -> [1, 1280, 8, 8]
        out_vae = self.mid_res0(out_vae, time)

        #[1, 1280, 8, 8],[1, 77, 768] -> [1, 1280, 8, 8]
        out_vae = self.mid_tf(out_vae, out_encoder)

        #[1, 1280, 8, 8],[1, 1280] -> [1, 1280, 8, 8]
        out_vae = self.mid_res1(out_vae, time)

        #----up----
        #[1, 1280+1280, 8, 8],[1, 1280] -> [1, 1280, 8, 8]
        out_vae = self.up_res0(torch.cat([out_vae, out_down.pop()], dim=1),
                               time)

        #[1, 1280+1280, 8, 8],[1, 1280] -> [1, 1280, 8, 8]
        out_vae = self.up_res1(torch.cat([out_vae, out_down.pop()], dim=1),
                               time)

        #[1, 1280+1280, 8, 8],[1, 1280] -> [1, 1280, 8, 8]
        out_vae = self.up_res2(torch.cat([out_vae, out_down.pop()], dim=1),
                               time)

        #[1, 1280, 8, 8] -> [1, 1280, 16, 16]
        out_vae = self.up_in(out_vae)

        #[1, 1280, 16, 16],[1, 77, 768],[1, 1280] -> [1, 1280, 32, 32]
        #out_down -> [1, 640, 16, 16],[1, 1280, 16, 16],[1, 1280, 16, 16]
        out_vae = self.up_block0(out_vae=out_vae,
                                 out_encoder=out_encoder,
                                 time=time,
                                 out_down=out_down)

        #[1, 1280, 32, 32],[1, 77, 768],[1, 1280] -> [1, 640, 64, 64]
        #out_down -> [1, 320, 32, 32],[1, 640, 32, 32],[1, 640, 32, 32]
        out_vae = self.up_block1(out_vae=out_vae,
                                 out_encoder=out_encoder,
                                 time=time,
                                 out_down=out_down)

        #[1, 640, 64, 64],[1, 77, 768],[1, 1280] -> [1, 320, 64, 64]
        #out_down -> [1, 320, 64, 64],[1, 320, 64, 64],[1, 320, 64, 64]
        out_vae = self.up_block2(out_vae=out_vae,
                                 out_encoder=out_encoder,
                                 time=time,
                                 out_down=out_down)

        #----out----
        #[1, 320, 64, 64] -> [1, 4, 64, 64]
        out_vae = self.out(out_vae)

        return out_vae


UNet()(torch.randn(2, 4, 64, 64), torch.randn(2, 77, 768),
    torch.LongTensor([26])).shape

torch.Size([2, 4, 64, 64])

加载预训练参数

In [13]:
from diffusers import UNet2DConditionModel

#加载预训练模型的参数
params = UNet2DConditionModel.from_pretrained(
    'lansinuote/diffsion_from_scratch.params', subfolder='unet')

unet = UNet()

#in
unet.in_vae.load_state_dict(params.conv_in.state_dict())
unet.in_time[0].load_state_dict(params.time_embedding.linear_1.state_dict())
unet.in_time[2].load_state_dict(params.time_embedding.linear_2.state_dict())


#down
def load_tf(model, param):
    model.norm_in.load_state_dict(param.norm.state_dict())
    model.cnn_in.load_state_dict(param.proj_in.state_dict())

    model.atten1.q.load_state_dict(
        param.transformer_blocks[0].attn1.to_q.state_dict())
    model.atten1.k.load_state_dict(
        param.transformer_blocks[0].attn1.to_k.state_dict())
    model.atten1.v.load_state_dict(
        param.transformer_blocks[0].attn1.to_v.state_dict())
    model.atten1.out.load_state_dict(
        param.transformer_blocks[0].attn1.to_out[0].state_dict())

    model.atten2.q.load_state_dict(
        param.transformer_blocks[0].attn2.to_q.state_dict())
    model.atten2.k.load_state_dict(
        param.transformer_blocks[0].attn2.to_k.state_dict())
    model.atten2.v.load_state_dict(
        param.transformer_blocks[0].attn2.to_v.state_dict())
    model.atten2.out.load_state_dict(
        param.transformer_blocks[0].attn2.to_out[0].state_dict())

    model.fc0.load_state_dict(
        param.transformer_blocks[0].ff.net[0].proj.state_dict())

    model.fc1.load_state_dict(
        param.transformer_blocks[0].ff.net[2].state_dict())

    model.norm_atten0.load_state_dict(
        param.transformer_blocks[0].norm1.state_dict())
    model.norm_atten1.load_state_dict(
        param.transformer_blocks[0].norm2.state_dict())
    model.norm_act.load_state_dict(
        param.transformer_blocks[0].norm3.state_dict())

    model.cnn_out.load_state_dict(param.proj_out.state_dict())


def load_res(model, param):
    model.time[1].load_state_dict(param.time_emb_proj.state_dict())

    model.s0[0].load_state_dict(param.norm1.state_dict())
    model.s0[2].load_state_dict(param.conv1.state_dict())

    model.s1[0].load_state_dict(param.norm2.state_dict())
    model.s1[2].load_state_dict(param.conv2.state_dict())

    if isinstance(model.res, torch.nn.Module):
        model.res.load_state_dict(param.conv_shortcut.state_dict())


def load_down_block(model, param):
    load_tf(model.tf0, param.attentions[0])
    load_tf(model.tf1, param.attentions[1])

    load_res(model.res0, param.resnets[0])
    load_res(model.res1, param.resnets[1])

    model.out.load_state_dict(param.downsamplers[0].conv.state_dict())


load_down_block(unet.down_block0, params.down_blocks[0])
load_down_block(unet.down_block1, params.down_blocks[1])
load_down_block(unet.down_block2, params.down_blocks[2])

load_res(unet.down_res0, params.down_blocks[3].resnets[0])
load_res(unet.down_res1, params.down_blocks[3].resnets[1])

#mid
load_tf(unet.mid_tf, params.mid_block.attentions[0])
load_res(unet.mid_res0, params.mid_block.resnets[0])
load_res(unet.mid_res1, params.mid_block.resnets[1])

#up
load_res(unet.up_res0, params.up_blocks[0].resnets[0])
load_res(unet.up_res1, params.up_blocks[0].resnets[1])
load_res(unet.up_res2, params.up_blocks[0].resnets[2])
unet.up_in[1].load_state_dict(
    params.up_blocks[0].upsamplers[0].conv.state_dict())


def load_up_block(model, param):
    load_tf(model.tf0, param.attentions[0])
    load_tf(model.tf1, param.attentions[1])
    load_tf(model.tf2, param.attentions[2])

    load_res(model.res0, param.resnets[0])
    load_res(model.res1, param.resnets[1])
    load_res(model.res2, param.resnets[2])

    if isinstance(model.out, torch.nn.Module):
        model.out[1].load_state_dict(param.upsamplers[0].conv.state_dict())


load_up_block(unet.up_block0, params.up_blocks[1])
load_up_block(unet.up_block1, params.up_blocks[2])
load_up_block(unet.up_block2, params.up_blocks[3])

#out
unet.out[0].load_state_dict(params.conv_norm_out.state_dict())
unet.out[2].load_state_dict(params.conv_out.state_dict())

Cannot initialize model with low cpu memory usage because `accelerate` was not found in the environment. Defaulting to `low_cpu_mem_usage=False`. It is strongly recommended to install `accelerate` for faster and less memory-intense model loading. You can do so with: 
```
pip install accelerate
```
.


Downloading:   0%|          | 0.00/3.44G [00:00<?, ?B/s]

KeyboardInterrupt: 

In [8]:
# out_vae = torch.randn(1, 4, 64, 64)
# out_encoder = torch.randn(1, 77, 768)
# time = torch.LongTensor([26])

# a = unet(out_vae=out_vae, out_encoder=out_encoder, time=time)
# b = params(out_vae, time, out_encoder).sample

# (a == b).all()